## Commissioner Equi-Join's Toughest Case (5 Points)
Copyright Jens Dittrich, Christian Schön & Jors Nix, [Big Data Analytics Group](https://bigdata.uni-saarland.de/), [CC-BY-SA](https://creativecommons.org/licenses/by-sa/4.0/legalcode)

In this exercise you will help commissioner equi-join solving one of his old, unsolved cases.

In [ ]:
import duckdb

## Load Data

Before we can start analyzing the data, we first have to load data from the corresponding csv files into an appropriate database schema. This is fake data.

In [ ]:
duckdb.sql("""
    CREATE TABLE households (
    id INTEGER PRIMARY KEY,
    street VARCHAR,
    postcode INTEGER,
    city VARCHAR,
    floor INTEGER
);""")

duckdb.sql("""
CREATE TABLE citizens (
    id INTEGER PRIMARY KEY,
    firstname VARCHAR,
    lastname VARCHAR,
    birthday TIMESTAMP
);""")

duckdb.sql("""
CREATE TABLE live_in (
    household_id INTEGER,
    citizen_id INTEGER,
    start TIMESTAMP,
    until TIMESTAMP,
    FOREIGN KEY(household_id) REFERENCES households(id),
    FOREIGN KEY(citizen_id) REFERENCES citizens(id),
    PRIMARY KEY(citizen_id, start)
);""")

duckdb.sql("""
CREATE TABLE articles (
    id INTEGER PRIMARY KEY,
    label VARCHAR,
    unit VARCHAR
);""")

duckdb.sql("""
CREATE TABLE groceries (
    id INTEGER PRIMARY KEY,
    caloriesPer100g INTEGER,
    FOREIGN KEY(id) REFERENCES articles(id)
);""")

duckdb.sql("""
CREATE TABLE purchases (
    article_id INTEGER,
    citizen_id INTEGER,
    date TIMESTAMP,
    amount FLOAT,
    FOREIGN KEY(article_id) REFERENCES articles(id),
    FOREIGN KEY(citizen_id) REFERENCES citizens(id),
    PRIMARY KEY(article_id, citizen_id, date)
);""")

In [ ]:
duckdb.sql(r"COPY households FROM '/content/households_no_header.csv' (FORMAT CSV, DELIMITER ',');")
duckdb.sql(r"COPY citizens FROM '/content/citizens_no_header.csv' (FORMAT CSV, DELIMITER ',');")
duckdb.sql(r"COPY live_in FROM '/content/live_in_no_header.csv' (FORMAT CSV, DELIMITER ',');")
duckdb.sql(r"COPY articles FROM '/content/articles_no_header.csv' (FORMAT CSV, DELIMITER ',');")
duckdb.sql(r"COPY groceries FROM '/content/groceries_no_header.csv' (FORMAT CSV, DELIMITER ',');")
duckdb.sql(r"COPY purchases FROM '/content/purchases_no_header.csv' (FORMAT CSV, DELIMITER ',');")

### Your query

Enter your query in the following cell. It should output the list of main suspects in the following format:
1. The suspects' first names as 'First_Name'
2. The last names of the suspects as 'Last_Name'

You are allowed to use subqueries and views.

In [ ]:
import duckdb

# Initialize an in-memory DuckDB instance
con = duckdb.connect()

# Create households table
con.execute("""
CREATE TABLE households (
    id INTEGER PRIMARY KEY,
    street VARCHAR,
    postcode INTEGER,
    city VARCHAR,
    floor INTEGER
);
""")

# Create citizens table
con.execute("""
CREATE TABLE citizens (
    id INTEGER PRIMARY KEY,
    firstname VARCHAR,
    lastname VARCHAR,
    birthday TIMESTAMP
);
""")

# Create live_in table
con.execute("""
CREATE TABLE live_in (
    household_id INTEGER,
    citizen_id INTEGER,
    start TIMESTAMP,
    until TIMESTAMP,
    FOREIGN KEY(household_id) REFERENCES households(id),
    FOREIGN KEY(citizen_id) REFERENCES citizens(id),
    PRIMARY KEY(citizen_id, start)
);
""")

# Create articles table
con.execute("""
CREATE TABLE articles (
    id INTEGER PRIMARY KEY,
    label VARCHAR,
    unit VARCHAR
);
""")

# Create groceries table
con.execute("""
CREATE TABLE groceries (
    id INTEGER PRIMARY KEY,
    caloriesPer100g INTEGER,
    FOREIGN KEY(id) REFERENCES articles(id)
);
""")

# Create purchases table
con.execute("""
CREATE TABLE purchases (
    article_id INTEGER,
    citizen_id INTEGER,
    date TIMESTAMP,
    amount FLOAT,
    FOREIGN KEY(article_id) REFERENCES articles(id),
    FOREIGN KEY(citizen_id) REFERENCES citizens(id),
    PRIMARY KEY(article_id, citizen_id, date)
);
""")

# Load the data from CSV files (assuming you have uploaded the files to Colab)
con.execute(r"COPY households FROM '/content/households_no_header.csv' (FORMAT CSV, DELIMITER ',');")
con.execute(r"COPY citizens FROM '/content/citizens_no_header.csv' (FORMAT CSV, DELIMITER ',');")
con.execute(r"COPY live_in FROM '/content/live_in_no_header.csv' (FORMAT CSV, DELIMITER ',');")
con.execute(r"COPY articles FROM '/content/articles_no_header.csv' (FORMAT CSV, DELIMITER ',');")
con.execute(r"COPY groceries FROM '/content/groceries_no_header.csv' (FORMAT CSV, DELIMITER ',');")
con.execute(r"COPY purchases FROM '/content/purchases_no_header.csv' (FORMAT CSV, DELIMITER ',');")


In [ ]:
# Create relevant_households view
con.execute("""
CREATE OR REPLACE VIEW relevant_households AS
SELECT id
FROM households
WHERE street LIKE '%13%'
   OR street LIKE '%bucht%'
   OR street LIKE '%Kor%';
""")

# Create residents_on_murder_date view
con.execute("""
CREATE OR REPLACE VIEW residents_on_murder_date AS
SELECT li.citizen_id
FROM live_in li
JOIN relevant_households rh ON li.household_id = rh.id
WHERE li.start <= '1943-11-24 15:00:00'
  AND (li.until IS NULL OR li.until >= '1943-11-24 15:00:00');
""")

# Create relevant_purchases view
con.execute("""
CREATE OR REPLACE VIEW relevant_purchases AS
SELECT p.citizen_id, p.article_id, p.amount, p.date
FROM purchases p
JOIN articles a ON p.article_id = a.id
WHERE a.label IN ('carrots', 'apples', 'onions')
  AND p.date BETWEEN '1943-11-19 15:00:00' AND '1943-11-24 15:00:00';
""")

# Create aggregated_purchases view
con.execute("""
CREATE OR REPLACE VIEW aggregated_purchases AS
SELECT
    rp.citizen_id,
    SUM(CASE WHEN a.label = 'carrots' THEN rp.amount ELSE 0 END) AS total_carrots,
    SUM(CASE WHEN a.label = 'apples' THEN rp.amount ELSE 0 END) AS total_apples,
    SUM(CASE WHEN a.label = 'onions' THEN rp.amount ELSE 0 END) AS total_onions
FROM relevant_purchases rp
JOIN articles a ON rp.article_id = a.id
GROUP BY rp.citizen_id
HAVING total_carrots = 0.5
   AND total_apples >= 2
   AND total_onions BETWEEN 1 AND 3;
""")

# Create suspects view
con.execute("""
CREATE OR REPLACE VIEW suspects AS
SELECT c.firstname AS First_Name, c.lastname AS Last_Name
FROM aggregated_purchases ap
JOIN citizens c ON ap.citizen_id = c.id
JOIN residents_on_murder_date romd ON ap.citizen_id = romd.citizen_id;
""")

# Query the suspects view to get the final result
suspects = con.execute("SELECT * FROM suspects;").fetchall()

# Display the suspects
for suspect in suspects:
    print(f"First Name: {suspect[0]}, Last Name: {suspect[1]}")


Error: Cannot drop entry "relevant_households" because there are entries that depend on it.
view "suspects" depends on view "relevant_households".
view "residents_on_murder_date" depends on view "relevant_households".
view "suspects" depends on view "residents_on_murder_date".
Use DROP...CASCADE to drop all dependents.

In [ ]:
con.execute("""DROP CASCADE""")